# **YOLO11_SMALL_MODEL_IMPLEMENTATION and different tracking techbiques**

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Libraries

In [ ]:
import os
import random
import shutil
import glob
from IPython.display import Image, display
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch

# Dataset Details

In [ ]:
import os

# Define the folder path
folder_path = '/content/drive/MyDrive/MLR/S3R2-KIIT'
files = os.listdir(folder_path)

for file in files:
    print(file)

In [ ]:
import os

# Define the paths for train, valid, and test directories
base_dir = '/content/drive/MyDrive/MLR/S3R2-KIIT'
dirs = ['train', 'valid', 'test']

for dir_name in dirs:
    dir_path = os.path.join(base_dir, dir_name)
    print(f"\nDirectory: {dir_name}")
    for subfolder in os.listdir(dir_path):
        subfolder_path = os.path.join(dir_path, subfolder)

        if os.path.isdir(subfolder_path):
            num_files = len([f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))])
            print(f"  Subfolder: {subfolder}, Number of files: {num_files}")


We need to change yml file path (ex. train: drive/MyDrive/MLR/S3R2-KIIT/train/images) information according to which location we stored the dataset in our drive.

In [ ]:
yaml_file_path = '/content/drive/MyDrive/MLR/S3R2-KIIT/S3R2_data.yml'
with open(yaml_file_path, 'r') as file:
    yaml_content = file.read()

print(yaml_content)


Let's see the information of a annotation file...

In [ ]:
import os

label_dir = '/content/drive/MyDrive/MLR/S3R2-KIIT/test/labels'
label_file = 'image_s3r2_kiit_116.txt'

label_path = os.path.join(label_dir, label_file)

if not os.path.exists(label_path):
    print(f"Label file not found: {label_path}")
else:
    with open(label_path, 'r') as f:
        lines = f.readlines()

    print(f"Label file: {label_file}")
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())

        print(f"Class ID: {int(class_id)}")
        print(f"Bounding Box: x_center={x_center}, y_center={y_center}, width={width}, height={height}")
        print("-" * 50)

In [ ]:
# Class names as per our dataset
class_names = [
    "Artilary",             # ID 0
    "Missile",              # ID 1
    "Radar",                # ID 2
    "M. Rocket Launcher",   # ID 3
    "Soldier",              # ID 4
    "Tank",                 # ID 5
    "Vehicle"               # ID 6
]

# Visualization

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Class names as per your dataset
class_names = [
    "Artilary",             # ID 0
    "Missile",              # ID 1
    "Radar",                # ID 2
    "M. Rocket Launcher",   # ID 3
    "Soldier",              # ID 4
    "Tank",                 # ID 5
    "Vehicle"               # ID 6
]

image_dir = '/content/drive/MyDrive/MLR/S3R2-KIIT/test/images'
annotation_dir = '/content/drive/MyDrive/MLR/S3R2-KIIT/test/labels'

def visualize_image_with_annotations(image_name):
    image_path = os.path.join(image_dir, image_name)
    img = cv2.imread(image_path)

    if img is None:
        print(f"Image not found: {image_path}")
        return

    annotation_name = os.path.splitext(image_name)[0] + '.txt'
    annotation_path = os.path.join(annotation_dir, annotation_name)

    if not os.path.exists(annotation_path):
        print(f"Annotation not found: {annotation_path}")
        return

    img_height, img_width = img.shape[:2]

    with open(annotation_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        # Parse values and convert class_id to integer
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        class_id = int(class_id)  # Ensure class_id is an integer

        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)

        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=(0, 255, 0), thickness=2)

        # Check if class_id is within bounds and retrieve class name
        if 0 <= class_id < len(class_names):
            class_name = class_names[class_id]
            cv2.putText(img, class_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        else:
            print(f"Class ID {class_id} is out of bounds for class names list.")

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 8))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

visualize_image_with_annotations('image_s3r2_kiit_1297.jpeg')  # Image to Visualize


In [ ]:
visualize_image_with_annotations('image_s3r2_kiit_150.jpeg')

In [ ]:
visualize_image_with_annotations('image_s3r2_kiit_1591.jpeg')

# Cheking for GPU

In [ ]:
if torch.cuda.is_available():
    gpu_info = torch.cuda.get_device_properties(0)
    print(f"GPU is available: {gpu_info.name}")
    print(f"Total Memory: {gpu_info.total_memory / (1024**3):.2f} GB")
    print(f"CUDA Capability: {gpu_info.major}.{gpu_info.minor}")
else:
    print("GPU not available, running on CPU.")


In [ ]:
!nvidia-smi

# Setting up YOLOV11-SMALL...

In [ ]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

Using the YOLOv8_SMALL model

In [ ]:
!yolo task=detect mode=train model=/content/yolo11s.pt data=/content/drive/MyDrive/MLR/S3R2-KIIT/S3R2_data.yml epochs=100 imgsz=640 plots=True

# Saving the result

In [ ]:
from google.colab import drive
import shutil

source_path = '/content/runs'
destination_path = '/content/drive/MyDrive/MLR/Yolo11Small_output'
shutil.copytree(source_path, destination_path)

print(f"The `runs` folder has been copied to: {destination_path}")

# Result Analysis

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and display the results image
results_image_path = '/content/drive/MyDrive/MLR/S3R2-KIIT_11SMALL/detect/train/results.png'
img = mpimg.imread(results_image_path)

plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.title('YOLO Model Training Results')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and display the F1 curve image
f1_curve_path = '/content/drive/MyDrive/MLR/S3R2-KIIT_11SMALL/detect/train/F1_curve.png'
img = mpimg.imread(f1_curve_path)

plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.title('F1 Score Curve')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Path to the confusion matrix image
image_path = '/content/drive/MyDrive/MLR/S3R2-KIIT_11SMALL/detect/train/confusion_matrix.png'

# Read and display the image
img = mpimg.imread(image_path)
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis('off')  # Hide the axes
plt.title('Confusion Matrix')
plt.show()


# Detection

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

image_folder = 'runs/detect/predict4/'

image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.webp'))]


for image_file in image_files:
    img_path = os.path.join(image_folder, image_file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.title(image_file)
    plt.axis('off')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and display the F1 curve image
f1_curve_path = '/content/runs/detect/train/F1_curve.png'
img = mpimg.imread(f1_curve_path)

plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.title('F1 Score Curve')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and display the results image
results_image_path = '/content/runs/detect/train/results.png'
img = mpimg.imread(results_image_path)

plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.title('YOLO Model Training Results')
plt.show()


In [ ]:

#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/runs/detect/predict5/missile.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:

#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/runs/detect/predict8/soldier.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:

#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/runs/detect/predict9/soldier2.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Tracking

In [ ]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics
!pip install -e .

In [ ]:
!yolo track model='/content/runs/detect/train/weights/best.pt' source='/content/downloaded_video7.mp4' save=True #'/content/good tank moving.mp4''/content/download.mp4'

In [ ]:
!ffmpeg -i "/content/runs/detect/track5/downloaded_video7.avi" "/content/runs/detect/track5/downloaded_video7.mp4"


In [ ]:
from IPython.display import Video

# Display the converted video
Video('/content/runs/detect/track/good_tank_moving.mp4', embed=True)


In [ ]:
from IPython.display import Video

# Display the converted video
Video('/content/runs/detect/track3/downloaded_video3.mp4', embed=True)


In [ ]:
from IPython.display import Video

# Display the converted video
Video('/content/runs/detect/track4/downloaded_video5.mp4', embed=True)


In [ ]:
from IPython.display import Video

# Display the converted video
Video('/content/runs/detect/track5/downloaded_video7.mp4', embed=True)


In [ ]:
from IPython.display import Video

# Display the converted video
Video('/content/runs/detect/track2/download.mp4', embed=True)


#DeepSort


In [ ]:
!pip install ultralytics
!pip install deep-sort-realtime

In [ ]:
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/MLR/S3R2-KIIT_11SMALL/detect/train/weights/best.pt')
class_names = ["Artillery", "Missile", "Radar", "M. Rocket Launcher", "Soldier", "Tank", "Vehicle"]

In [ ]:
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/MLR/S3R2-KIIT_11SMALL/detect/train/weights/best.pt')
tracker = DeepSort(max_age=30, n_init=2, max_iou_distance=0.7)

class_names = ["Artillery", "Missile", "Radar", "M. Rocket Launcher", "Soldier", "Tank", "Vehicle"]

input_video_path = "/content/Tracking_Test.mp4"
output_video_path = "/content/Tracking_Test_DeepSort_Tracked.avi"

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*"XVID")  # Codec for AVI files
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, save=False, save_txt=False, conf=0.5)

    detections = []
    class_ids = []  # List to store class IDs for each detection
    for result in results[0].boxes:
        # Convert (x1, y1, x2, y2) to (x, y, width, height)
        x1, y1, x2, y2 = result.xyxy[0].tolist()
        x, y = x1, y1
        width, height = x2 - x1, y2 - y1
        confidence = result.conf[0].item()
        class_id = int(result.cls[0].item())

        # Append detection in the format [(x, y, width, height), confidence, class_id]
        detections.append([(x, y, width, height), confidence, class_id])
        class_ids.append(class_id)  # Save the class_id for use with tracking results

    tracked_objects = tracker.update_tracks(detections, frame=frame)

    for track in tracked_objects:
        if not track.is_confirmed():
            continue
        ltrb = track.to_ltrb()  # Left-Top-Right-Bottom coordinates

        class_id = track.det_class
        class_name = class_names[class_id] if class_id < len(class_names) else "Unknown"

        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 255, 0), 2)
        cv2.putText(frame, class_name, (int(ltrb[0]), int(ltrb[1] - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Write the frame to the output video file
    out.write(frame)

cap.release()
out.release()
print("Tracking complete. Output saved to:", output_video_path)

In [ ]:
!pip install pydub
!apt-get install ffmpeg

In [ ]:
!ffmpeg -i "/content/Tracking_Test_DeepSort_Tracked.avi" "/content/Tracking_Test_DeepSort_Tracked.mp4"

In [ ]:
from IPython.display import Video

# Display the converted video (AVI format)
Video('/content/Tracking_Test_DeepSort_Tracked.mp4', embed=True)

In [ ]:
from IPython.display import Video

# Display the converted video (AVI format)
Video('/content/downloaded_video5_tracked.mp4', embed=True)

In [ ]:
from IPython.display import Video

# Display the converted video (AVI format)
Video('/content/downloaded_video7_tracked.mp4', embed=True)

# Botsort

In [ ]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics
!pip install -e .

In [ ]:
!yolo track model='/content/drive/MyDrive/MLR/S3R2-KIIT_11SMALL/detect/train/weights/best.pt' \
            source='/content/Tracking_Test.mp4' \
            save=True \
            tracker='/content/ultralytics/ultralytics/cfg/trackers/botsort.yaml'

In [ ]:
!ffmpeg -i "/content/ultralytics/runs/detect/track/Tracking_Test.avi" "/content/ultralytics/runs/detect/track/Tracking_Test.mp4"

In [ ]:
from IPython.display import Video

# Display the converted video
Video('/content/ultralytics/runs/detect/track/Tracking_Test.mp4', embed=True)


# Byte Track

In [ ]:
!yolo track model='/content/drive/MyDrive/MLR/S3R2-KIIT_11SMALL/detect/train/weights/best.pt' \
            source='/content/Tracking_Test.mp4' \
            save=True \
            tracker='/content/ultralytics/ultralytics/cfg/trackers/bytetrack.yaml'

In [ ]:
!ffmpeg -i "/content/ultralytics/runs/detect/track2/Tracking_Test.avi" "/content/ultralytics/runs/detect/track2/Tracking_Test.mp4"

In [ ]:
from IPython.display import Video

# Display the converted video
Video('/content/ultralytics/runs/detect/track2/Tracking_Test.mp4', embed=True)

In [ ]:
!yolo track model='/content/drive/MyDrive/MLR/S3R2-KIIT_11SMALL/detect/train/weights/best.pt' \
      source='/content/downloaded_video7.mp4' \
      save=True \
      save_txt=True \
      tracker='/content/ultralytics/ultralytics/cfg/trackers/bytetrack.yaml'

In [ ]:
!ffmpeg -i "/content/ultralytics/runs/detect/track4/downloaded_video7.avi" "/content/ultralytics/runs/detect/track4/downloaded_video7_Byte_Tracked.mp4"

In [ ]:
from IPython.display import Video

# Display the converted video
Video('/content/ultralytics/runs/detect/track4/downloaded_video7_Byte_Tracked.mp4', embed=True)